So, the new popular thing around NLP has been these word embeddings and even though this is a new topic for me, I thought I'd share what I'd learnt and get to know more in doing so.

I am using the gensim library in python which does not seem to have a very good documentation or tutorial series. What I have done below is a compilation from various sources and I think this might work for something I am trying right now.

Please do write out your views and comment so I can improve.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import nltk
from nltk import word_tokenize
import re

from sklearn.model_selection import train_test_split as tts
from gensim.models import doc2vec
import gensim
import json

import sys

STOP_WORDS = nltk.corpus.stopwords.words()


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Okay, learning from my previous venture [Cosine Similarity using TFIDF Weighting](https://www.kaggle.com/antriksh5235/quora-question-pairs/cosine-similarity-using-tfidf-weighting) I decided it was best to drop the questions less than a length of 10 and the stop words in sentences in order to avoid a lot of computation.

The following function was written to take care of all of this.

In [ ]:
def clean_sentence(sent):
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', sent).lower()
    sentence = sentence.split(" ")

    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)

    sentence = " ".join(sentence)
    return sentence

I am still figuring out how I need to be able to find similarity between two document sets. So, I am keeping an implementation of cosine similarity (between two 1D vectors, just in case.)

In [ ]:
import math
import numpy as np

def cosine(v1, v2):
    """
            v1 and v2 are two vectors (can be list of numbers) of the same dimensions. Function returns the cosine distance between those
            which is the ratio of the dot product of the vectors over their RS.
    """
    v1 = np.array(v1)
    v2 = np.array(v2)

    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

This one is simple enough, just dropping all the nans.

In [ ]:
data = pd.read_csv('../input/train.csv')
data = data.dropna(how="any")

I was first going for merging the two sentence sets into one so that the system is trained over all the words/sentences/document vectors and I would not be able to do it without concatenating both of these.

In [ ]:
def concatenate(data):
    X_set1 = data['question1']
    X_set2 = data['question2']
#    y = data['is_duplicate']
    X = X_set1.append(X_set2, ignore_index=True)
    
    return X

The above method is being run here to clean the sentences one by one.

In [ ]:
print('Cleaning data, this might take long')
for col in ['question1', 'question2']:
    data[col] = data[col].apply(clean_sentence)

I am splitting into training and testing set for now. The test given with this competition is very large and I only want to use it once I have tested out implementations.

In [ ]:
print('Splitting data to train and test sets.')
y = data['is_duplicate']
X_train, X_test, y_train, y_test = tts(data[['id','question1', 'question2']], y, test_size=0.3)

X_train.head()

Like I said, I am compiling my understanding of gensim from a lot of sources and one of them used multiprocessing, stating that it might be painfully slow doing otherwise.

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1

This is where the initial usage of gensim begins. Notice that I am yielding output from the __iter__ method, which is actually why I wrote this modification of the gensim LabeledLineSentence class. Every question is a document and every document has to be tagged, which is where I am using the id. I might as well start using the question Ids once I figure out how I am going to use the or I'll just append a '_q1' and '_q2' for each to know which is which and compare the same set of questions for every ID.

Once done, I hope this somehow helps in boosting performance. The only reason I went for doc2vec instead of word2vec was it might be able to capture the semantic relations within the sentence/question.

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models import doc2vec
class LabeledLineSentence(object):

    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.TaggedDocument(words=word_tokenize(doc),
                                         tags=[self.labels_list[idx]])

The iterator returns a yield of a TaggedDocument every time the Doc2Vec.build_vocab() function requests it. Had I not given the iterator with the Doc2Vec, I would have to call another model1.build_vocab(it) function just to perform the initialisation. This seemed a quick getaway from writing more lines.

In [ ]:
X = concatenate(X_train)
labels = []
for label in X_train['id'].tolist():
    labels.append('SENT_%s_1' % label)
for label in X_train['id'].tolist():
    labels.append('SENT_%s_2' % label)

docs = LabeledLineSentence(X.tolist(), labels)
it = docs.__iter__()
model1 = Doc2Vec(it, size=12, window=8, min_count=5, workers=4)

Okay, the more documents you have, the better. To better measure the similarity of documents, it is undoubtable that your model needs to have a very established vector for each.

Also, instead of running for the normal 10-20 epochs that people usually have for training Doc2vec models, I tried 100 epochs just in case. Let's see how the results turn up on the test sets I extracted.

In [ ]:
for epoch in range(10):
    model1.train(it)
    model1.alpha -= 0.0002  # decrease the learning rate
    model1.min_alpha = model1.alpha  # fix the learning rate, no deca
    model1.train(it)

Now finally for the similarity. I hope it turns out to be some good.

In [ ]:
X_test.index = np.arange(0, X_test['question1'].shape[0])
y_test.index = np.arange(0, X_test['question1'].shape[0])
#print(X_test)
count = 0
for i in range(X_test['question1'].shape[0]):
    doc1 = word_tokenize(X_test['question1'][i])
    doc2 = word_tokenize(X_test['question2'][i])

    docvec1 = model1.infer_vector(doc1)
    docvec2 = model1.infer_vector(doc2)

    #print(docvec1)
    #print(docvec2)

    print(cosine(docvec1, docvec2), y_test[i])
    if count>100:
        break
    count+=1